In [2]:
import requests
import pandas as pd

 
access_token = get_access_token()


# Specify the fields you want to retrieve
fields = 'text,create_time'

# Specify the video IDs for which you want to retrieve comments
video_ids = ['7362160166462098720','7362103251178687776','7361835619934309664','7361075678558653729','7358493419607428385',
             '7358474078660988192','7356993589970324769','7356977924748168480','7351044113455680800','7350330568120339745',
             '7348463505323887905','7345518842073877793','7343255198434872609','7341400236293704992','7333566508431445280',
             '7331294354012753185','7330625031325617440','7330623382658092321','7329924914318691616','7329817820865350944',
             '7327284769723600160','7325790805162036512'] # Add more video IDs as needed

# List to store all comments
all_data = []

# Iterate over each video ID
for video_id in video_ids:
    # Make the request to retrieve comments for the current video ID
    isFetchingComments = True
    cursor = 0
    MAX_COUNT = 100
    while isFetchingComments:
        response = requests.post(
            'https://open.tiktokapis.com/v2/research/video/comment/list/?fields=%s' % fields,
            headers={'authorization': 'bearer ' + access_token},
            data={ 
                'video_id': video_id,
                'max_count': MAX_COUNT,
                'cursor': cursor
            }
        )
        
        # Extract comments from the response
        data = response.json()
        comments = data.get('data', {}).get('comments', [])
        
        # Append comments to the list
        for comment in comments:
            comment['video_id'] = video_id
            all_data.append(comment)
        
        if len(comments) == 0:
            isFetchingComments = False
            
        else:
            cursor += MAX_COUNT

# Create DataFrame
df = pd.DataFrame(all_data)

print(df)

      create_time                                               text  \
0      1714372883    Nö mehr Mitglieder mehr % bei wahlprogrosen 💚💚💚   
1      1714322104                   bezahlen kann Mann das nur nicht   
2      1714286460  Super! Extrem wichtig das Klimapaket. Nur so s...   
3      1714317370                            Die grünen sind am Ende   
4      1714393905  gibt keinen Grund. warum für gute Arbeit zurüc...   
...           ...                                                ...   
6425   1706127665  Kein Gesetz wird "mal eben so" geändert. Dafür...   
6426   1705994030  Durch Populisten aufgegriffen wurden, um Leute...   
6427   1705743530           Weil ein paar Leute wieder gehen müssen?   
6428   1705735090  Das ist doch nicht normal, die Grünen werden i...   
6429   1713311510                einfach nur ein Lacher diese Frau😡😡   

                 video_id  
0     7362160166462098720  
1     7362160166462098720  
2     7362160166462098720  
3     73621601664620987

In [3]:
#### CONVERT DATE SO IT IS LATER READABLE IN R:

import pandas as pd

# Step 1: Create a sample DataFrame

df = pd.DataFrame(all_data)

# Step 2: Convert the Unix timestamps to human-readable date and time
df['create_time_readable'] = pd.to_datetime(df['create_time'], unit='s')

# Display the DataFrame
print(df)

      create_time                                               text  \
0      1714372883    Nö mehr Mitglieder mehr % bei wahlprogrosen 💚💚💚   
1      1714322104                   bezahlen kann Mann das nur nicht   
2      1714286460  Super! Extrem wichtig das Klimapaket. Nur so s...   
3      1714317370                            Die grünen sind am Ende   
4      1714393905  gibt keinen Grund. warum für gute Arbeit zurüc...   
...           ...                                                ...   
6425   1706127665  Kein Gesetz wird "mal eben so" geändert. Dafür...   
6426   1705994030  Durch Populisten aufgegriffen wurden, um Leute...   
6427   1705743530           Weil ein paar Leute wieder gehen müssen?   
6428   1705735090  Das ist doch nicht normal, die Grünen werden i...   
6429   1713311510                einfach nur ein Lacher diese Frau😡😡   

                 video_id create_time_readable  
0     7362160166462098720  2024-04-29 06:41:23  
1     7362160166462098720  2024-04-28

In [6]:
#Check working directory
import os

# Check current working directory
print(os.getcwd())

# Save to the current working directory
df.to_csv('/Users/emma/Desktop/Gruene_comments.csv', index=False)

/


In [7]:
# This is our classification prompt
PROMPT =  "Conduct a sentiment analysis of the comments on TikTok Vidoes by the official social media profiles of German parliamentarian parties on TikTok. The comments are generated in German. Consider the tone, context, and language and emojis used in each comment to determine its sentiment.  Respond with '0' if the comment contains neutral sentiment, respond with '1' if the content contains positive sentiment, and '2' if the comment contains negative sentiments. No explanation is needed. Here are some examples to guide your analysis. The comment 'Das nenn ich mal eine gute Partei',  the sentiment should be categorized as positive, so 1. If the comment is 'Ich habe keine starken Gefühle zu diesem Thema', the sentiment should be catgeorized as neutral, so 0. An example for a negative sentiment comment is 'Wenn das böse ein Gesicht hat', so it should be categorizes as 2." 

In [8]:
import openai
from openai import OpenAI

In [9]:
#We define which model to use throughout
MODEL = 'gpt-3.5-turbo-0125' # We use the cheap model 
WAIT_TIME = 0.01 # Wait time between each request. This depends on the rate limit of the model used

client = OpenAI(
    api_key= "" #Set the API key. See the how-to guide for further instructions
)

In [10]:
# Simple function for analyzing a message with OpenAI
def analyze_message(text, temperature=0.2):
    try:
        response = client.chat.completions.create(
            model = MODEL, 
            temperature=temperature,
            messages=[
                    {"role": "system", "content": f"'{PROMPT}'"}, #The system instruction tells the bot how it is supposed to behave
                    {"role": "user", "content": f"'{text}'"} #This provides the text to be analyzed.
                ]
        )
        result = ''
        for choice in response.choices:
            result += choice.message.content
        print(f"{text}: {result}")
        return result 
    except Exception as e:
        # Not the prettiest solution; but easy
        print(f'Exception happened: {e}')
        return 'error'

In [11]:
analyze_message("'Euch würde ich nicht wählen!'")

'Euch würde ich nicht wählen!': 2


'2'

In [12]:
import pandas as pd

# Specify the path to the CSV file
file_path = '/Users/emma/Desktop/G_cleaned_data.csv'

# Load the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Display the first few rows of the DataFrame
print(df)

      create_time                                               text  \
0      1714372883    Nö mehr Mitglieder mehr % bei wahlprogrosen 💚💚💚   
1      1714322104                   bezahlen kann Mann das nur nicht   
2      1714286460  Super! Extrem wichtig das Klimapaket. Nur so s...   
3      1714317370                            Die grünen sind am Ende   
4      1714393905  gibt keinen Grund. warum für gute Arbeit zurüc...   
...           ...                                                ...   
5573   1706127665  Kein Gesetz wird "mal eben so" geändert. Dafür...   
5574   1705994030  Durch Populisten aufgegriffen wurden, um Leute...   
5575   1705743530           Weil ein paar Leute wieder gehen müssen?   
5576   1705735090  Das ist doch nicht normal, die Grünen werden i...   
5577   1713311510                einfach nur ein Lacher diese Frau😡😡   

                 video_id create_time_readable  
0     7362160166462099456  2024-04-29 06:41:23  
1     7362160166462099456  2024-04-28

In [13]:
# We start by defining a new column with the result
df['sentiment'] = None

# Save the DataFrame to a CSV file
df.to_csv('/Users/emma/SEN_G.csv', compression='gzip', index=False)

In [14]:
import time 
i = 0
while(True):

    #Find all unprocessed lines; where 
    left = df.loc[df['sentiment'].isna()]
    
    #No lines left? Then we're done
    if len(left)==0:
        print("All done!")
        break
        
    #Take a random line
    line = left.sample()
    index = line.index.values[0]
    text = line['text'].values[0]
    
    print(f"There are {len(left)} left to process. Processing: {index}")
    
    #Wait for a bit, to not overload the API
    time.sleep(WAIT_TIME)
    
    #Analyze the specific line, chunk by chunk    
    result = analyze_message(text)
    df.loc[index,'sentiment'] = result 

    i+=1    
    #Save the result to persistent file
    df.to_csv('/Users/emma/Sentiment_G.csv.gz',compression='gzip',index=False)

There are 5578 left to process. Processing: 3908
Sehr gute Reaktion, vielen Dank!: 1
There are 5577 left to process. Processing: 2272
AFD FÜR DEUTSCHLAND 🇩🇪🇩🇪🇩🇪🇩🇪🇩🇪🇩🇪🇩🇪🇩🇪: 2
There are 5576 left to process. Processing: 1312
Hat vorher auch niemand verhindert, es geht lediglich hier um die Dokumente: 2
There are 5575 left to process. Processing: 2862
Was veranlasst dich zu dieser Annahme?: 0
There are 5574 left to process. Processing: 5446
Was kostet es eigentlich so einen Ventilator zu entsorgen, wenn die Subventionen ausgelaufen sind?: 2
There are 5573 left to process. Processing: 4559
Danke Herr Habeck 👍💚💚: 1
There are 5572 left to process. Processing: 1702
🤭🤭🤭🤭🤭🤭: 0
There are 5571 left to process. Processing: 3727
Nana, Du musst doch nicht gleich in nervöses Kichern verfallen, lass Dir Zeit, lies ganz in Ruhe, dann klappt das schon mit dem Leseverständnis 🤗: 1
There are 5570 left to process. Processing: 4451
die macht auch nichts freiwilig, lässt es sich doch gut bezahlen😁: 2
There a

In [15]:
#Check if data is the way we want
print(df.head(10))

   create_time                                               text  \
0   1714372883    Nö mehr Mitglieder mehr % bei wahlprogrosen 💚💚💚   
1   1714322104                   bezahlen kann Mann das nur nicht   
2   1714286460  Super! Extrem wichtig das Klimapaket. Nur so s...   
3   1714317370                            Die grünen sind am Ende   
4   1714393905  gibt keinen Grund. warum für gute Arbeit zurüc...   
5   1714393339  co 2 ausgleich ist so eine abzocke\nentweder i...   
6   1714372994                                               😂😂😂😂   
7   1714295212  Und davon sind die meisten fast 10j alt u werd...   
8   1714299003                                               👏👏👏💚   
9   1714310691                        Die Grünen =Dumm und dümmer   

              video_id create_time_readable sentiment  
0  7362160166462099456  2024-04-29 06:41:23         1  
1  7362160166462099456  2024-04-28 16:35:04         2  
2  7362160166462099456  2024-04-28 06:41:00         1  
3  7362160166462

In [16]:
# Export to CSV
df.to_csv('/Users/emma/Green_done.csv',index=False)